# Прогнозирование с помощью регрессии
Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке!

Вам понадобится много признаков. Некоторые из них у вас уже есть — это:

- идентификатор географической зоны
- дата и время
- количество поездок в периоды, предшествующие прогнозируемому
- синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
- Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!

Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования $\hat{y}_{T+1|T}$, другая для $\hat{y}_{T+2|T} $  и т.д.

### Чтобы сдать задание, выполните следующую последовательность действий.

1. Для каждой из шести задач прогнозирования $\hat{y}_{T+i|T}, i=1,\dots,6 $,i=1,…,6 сформируйте выборки. Откликом будет $y_{T+i}y $  при всевозможных значениях T, а признаки можно использовать следующие:
- идентификатор географической зоны — категориальный
- год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
- синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
- сами значения прогнозов ARIMA $\hat{y}_{T+i|T}^{ARIMA} $
- количество поездок из рассматриваемого района в моменты времени $y_T, y_{T-1}, \dots, y_{T-K}$ (параметр K можно подбирать; попробуйте начать, например, с 6)
- количество поездок из рассматриваемого района в моменты времени $y_{T-24}, y_{T-48}, \dots, y_{T-24*K_d}$ (параметр $K_d$ можно подбирать; попробуйте начать, например, с 2)
- суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании $\hat{y}_{T+i|T}, i=1,\dots,6 $
,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.

2. Разбейте каждую из шести выборок на три части:

- обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
- тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
- итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

3. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

- допускала попарные взаимодействия между признаками
- была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)

4. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

$$Q_{may} =\frac1{R * 739* 6} \sum\limits_{r=1}^{R} \sum_{T=2016.04.30 23:00}^{2016.05.31 17:00} \sum_{i=1}^6 \left| \hat{y}^r_{T|T+i} - y^r_{T+i} \right|$$

Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу QQ, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

5. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

## Загрузка и предобработка данных

In [50]:
import pandas as pd
import numpy as np
import scipy as sc
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import model_selection
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.metrics import mean_absolute_error as mae
from itertools import product
import datetime

In [51]:
import xgboost as xgb

In [52]:
import warnings
warnings.simplefilter(action='ignore')

Загружаем список из 102 регионов для прогноза, отобранных на неделе 2

In [53]:
nonzero_reg=pd.read_csv("nonzero_regions.csv").iloc[:,1]

In [142]:
regions = nonzero_reg.values 

In [143]:
regions

array([1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132,
       1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182,
       1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230,
       1231, 1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280,
       1281, 1282, 1283, 1284, 1285, 1286, 1287, 1326, 1327, 1331, 1332,
       1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380,
       1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431,
       1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1482, 1483,
       1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069,
       2118, 2119, 2168], dtype=int64)

Загружаем аггрегированные данные за январь-июнь 2016. Отбираем для дальнейшего анализа данные только по отобранным 102 регионам

In [126]:
df_2=pd.read_csv("taxi_2016_02.csv")
df_1=pd.read_csv("taxi_2016_01.csv")
df_3=pd.read_csv("taxi_2016_03.csv")
df_4=pd.read_csv("taxi_2016_04.csv")
df_5=pd.read_csv("taxi_2016_05.csv")
df_6=pd.read_csv("taxi_2016_06.csv")
df=pd.concat([ df_1,df_2,df_3,df_4,df_5, df_6], axis=0)

In [127]:
df.rename(columns={'Unnamed: 0':'time'}, inplace=True)
df.index=df.time
df=df.iloc[:,nonzero_reg]  # отбираем только нужные регионы
df.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


Загружаем датасет с прогнозами ARIMA на январь-июнь, полученным на прошлой неделе

In [58]:
forecast=pd.read_csv('forecast.csv')
forecast.head()
forecast.index=forecast.time
forecast=forecast.drop(['time'], axis=1)
forecast.head()

,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6,region
time,,,,,,,
2016-01-01 00:00:00,4.376683,4.826494,3.196990,2.524114,4.825234,5.174734,1435
2016-01-01 01:00:00,19.951172,17.838943,16.698748,18.547465,18.459001,24.245501,1435
2016-01-01 02:00:00,41.120005,39.236746,40.366111,39.581274,44.693627,47.736560,1435
2016-01-01 03:00:00,27.503669,29.007512,28.585211,34.048519,37.431204,36.497186,1435
2016-01-01 04:00:00,20.775319,20.615761,26.333432,29.962355,29.266716,22.562023,1435


In [59]:
np.unique(forecast['region']).shape

(102,)

In [60]:
forecast.index

Index(['2016-01-01 00:00:00', '2016-01-01 01:00:00', '2016-01-01 02:00:00',
       '2016-01-01 03:00:00', '2016-01-01 04:00:00', '2016-01-01 05:00:00',
       '2016-01-01 06:00:00', '2016-01-01 07:00:00', '2016-01-01 08:00:00',
       '2016-01-01 09:00:00',
       ...
       '2016-06-30 09:00:00', '2016-06-30 10:00:00', '2016-06-30 11:00:00',
       '2016-06-30 12:00:00', '2016-06-30 13:00:00', '2016-06-30 14:00:00',
       '2016-06-30 15:00:00', '2016-06-30 16:00:00', '2016-06-30 17:00:00',
       '2016-06-30 18:00:00'],
      dtype='object', name='time', length=445026)

In [61]:
#df = df.iloc[:-5]

In [128]:
df

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,116.0,190.0,135.0,132.0,395.0,308.0,401.0,336.0,496.0,260.0,...,2.0,44.0,4.0,297.0,311.0,104.0,9.0,142.0,96.0,1.0
2016-06-30 20:00:00,104.0,142.0,149.0,141.0,333.0,368.0,390.0,385.0,560.0,247.0,...,1.0,27.0,7.0,288.0,344.0,103.0,24.0,209.0,145.0,0.0
2016-06-30 21:00:00,151.0,162.0,145.0,135.0,359.0,422.0,460.0,541.0,672.0,259.0,...,2.0,21.0,9.0,287.0,307.0,185.0,9.0,213.0,142.0,1.0


In [63]:
forecast.shape

(445026, 7)

## Формирование тренировочного, валидационного и тестового датасетов 

###  Задание 1


In [64]:
# Ваш код
#forecast = forecast.dropna()

In [65]:
forecast

,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6,region
time,,,,,,,
2016-01-01 00:00:00,4.376683,4.826494,3.196990,2.524114,4.825234,5.174734,1435
2016-01-01 01:00:00,19.951172,17.838943,16.698748,18.547465,18.459001,24.245501,1435
2016-01-01 02:00:00,41.120005,39.236746,40.366111,39.581274,44.693627,47.736560,1435
2016-01-01 03:00:00,27.503669,29.007512,28.585211,34.048519,37.431204,36.497186,1435
2016-01-01 04:00:00,20.775319,20.615761,26.333432,29.962355,29.266716,22.562023,1435
...,...,...,...,...,...,...,...
2016-06-30 14:00:00,1.405225,1.391145,1.216948,2.013569,1.631919,0.999939,1630
2016-06-30 15:00:00,1.140887,0.996133,1.920821,1.548399,0.924728,1.277560,1630
2016-06-30 16:00:00,0.971042,1.898682,1.539100,0.916354,1.270019,2.082705,1630


In [66]:
time = pd.to_datetime(forecast.index.values, format='%Y-%m-%d %H:%M:%S')

In [67]:
time

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 01:00:00',
               '2016-01-01 02:00:00', '2016-01-01 03:00:00',
               '2016-01-01 04:00:00', '2016-01-01 05:00:00',
               '2016-01-01 06:00:00', '2016-01-01 07:00:00',
               '2016-01-01 08:00:00', '2016-01-01 09:00:00',
               ...
               '2016-06-30 09:00:00', '2016-06-30 10:00:00',
               '2016-06-30 11:00:00', '2016-06-30 12:00:00',
               '2016-06-30 13:00:00', '2016-06-30 14:00:00',
               '2016-06-30 15:00:00', '2016-06-30 16:00:00',
               '2016-06-30 17:00:00', '2016-06-30 18:00:00'],
              dtype='datetime64[ns]', length=445026, freq=None)

In [68]:
data = pd.DataFrame()

In [69]:
data['day'] = time.day
data['month'] = time.month
data['day_of_week'] = time.day_of_week
data['hour'] = time.hour

In [70]:
t = np.arange(data.shape[0])
w = 168
for i in range(21):
    cos = 'c' + str(i+1)
    sin = 's' + str(i+1)
    data[cos] = np.cos(2*np.pi*i*t/w)
    data[sin] = np.sin(2*np.pi*i*t/w)

In [71]:
for i in range(6):
    col_name = 'y_f' + str(i+1)
    data_name = 'forecast_' + str(i+1)
    data[col_name] = forecast[data_name].values

In [72]:
#data['region'] = forecast['region'].values

In [73]:
data

,day,month,day_of_week,hour,c1,s1,c2,s2,c3,s3,...,c20,s20,c21,s21,y_f1,y_f2,y_f3,y_f4,y_f5,y_f6
0,1,1,4,0,1.0,0.0,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,4.376683,4.826494,3.196990,2.524114,4.825234,5.174734
1,1,1,4,1,1.0,0.0,0.999301,0.037391,0.997204,0.074730,...,0.757972,0.652287,0.733052,0.680173,19.951172,17.838943,16.698748,18.547465,18.459001,24.245501
2,1,1,4,2,1.0,0.0,0.997204,0.074730,0.988831,0.149042,...,0.149042,0.988831,0.074730,0.997204,41.120005,39.236746,40.366111,39.581274,44.693627,47.736560
3,1,1,4,3,1.0,0.0,0.993712,0.111964,0.974928,0.222521,...,-0.532032,0.846724,-0.623490,0.781831,27.503669,29.007512,28.585211,34.048519,37.431204,36.497186
4,1,1,4,4,1.0,0.0,0.988831,0.149042,0.955573,0.294755,...,-0.955573,0.294755,-0.988831,0.149042,20.775319,20.615761,26.333432,29.962355,29.266716,22.562023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445021,30,6,3,14,1.0,0.0,0.916562,-0.399892,0.680173,-0.733052,...,0.037391,-0.999301,-0.365341,-0.930874,1.405225,1.391145,1.216948,2.013569,1.631919,0.999939
445022,30,6,3,15,1.0,0.0,0.930874,-0.365341,0.733052,-0.680173,...,0.680173,-0.733052,0.365341,-0.930874,1.140887,0.996133,1.920821,1.548399,0.924728,1.277560
445023,30,6,3,16,1.0,0.0,0.943883,-0.330279,0.781831,-0.623490,...,0.993712,-0.111964,0.900969,-0.433884,0.971042,1.898682,1.539100,0.916354,1.270019,2.082705
445024,30,6,3,17,1.0,0.0,0.955573,-0.294755,0.826239,-0.563320,...,0.826239,0.563320,0.955573,0.294755,2.081930,1.700789,0.984268,1.331176,2.137778,2.261178


In [74]:
regions = forecast['region']

In [75]:
data_len = 445026

In [76]:
y_all = []
for reg in forecast['region'].unique():
    y_all += list(df.iloc[:-5][str(int(reg))].values)

In [77]:
print(len(y_all))

445026


In [78]:
data['y_t'] = y_all[0:data_len]
data['y_t-1'] = [np.nan]*1 + y_all[0:data_len - 1]
data['y_t-2'] = [np.nan]*2 + y_all[0:data_len - 2]
data['y_t-3'] = [np.nan]*3 + y_all[0:data_len - 3]
data['y_t-4'] = [np.nan]*4 + y_all[0:data_len - 4]
data['y_t-5'] = [np.nan]*5 + y_all[0:data_len - 5]
data['y_t-6'] = [np.nan]*6 + y_all[0:data_len - 6]

In [79]:
data['y_t-24'] = [np.nan]*24 + y_all[0:data_len - 24]
data['y_t-48'] = [np.nan]*48 + y_all[0:data_len - 48]
data['y_t-72'] = [np.nan]*72 + y_all[0:data_len - 72]
data['y_t-96'] = [np.nan]*96 + y_all[0:data_len - 96]
data['y_t-120'] = [np.nan]*120 + y_all[0:data_len - 120]
data['y_t-144'] = [np.nan]*144 + y_all[0:data_len - 144]
data['y_t-168'] = [np.nan]*168 + y_all[0:data_len - 168]

In [80]:
#data = data.dropna()

In [81]:
data

,day,month,day_of_week,hour,c1,s1,c2,s2,c3,s3,...,y_t-4,y_t-5,y_t-6,y_t-24,y_t-48,y_t-72,y_t-96,y_t-120,y_t-144,y_t-168
0,1,1,4,0,1.0,0.0,1.000000,0.000000,1.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,4,1,1.0,0.0,0.999301,0.037391,0.997204,0.074730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,4,2,1.0,0.0,0.997204,0.074730,0.988831,0.149042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,4,3,1.0,0.0,0.993712,0.111964,0.974928,0.222521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,4,4,1.0,0.0,0.988831,0.149042,0.955573,0.294755,...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445021,30,6,3,14,1.0,0.0,0.916562,-0.399892,0.680173,-0.733052,...,0.0,4.0,3.0,0.0,2.0,1.0,5.0,4.0,3.0,5.0
445022,30,6,3,15,1.0,0.0,0.930874,-0.365341,0.733052,-0.680173,...,0.0,0.0,4.0,1.0,1.0,1.0,4.0,4.0,1.0,8.0
445023,30,6,3,16,1.0,0.0,0.943883,-0.330279,0.781831,-0.623490,...,0.0,0.0,0.0,2.0,4.0,1.0,3.0,4.0,2.0,2.0
445024,30,6,3,17,1.0,0.0,0.955573,-0.294755,0.826239,-0.563320,...,1.0,0.0,0.0,4.0,0.0,1.0,1.0,2.0,5.0,1.0


In [82]:
reg_OHE =pd.get_dummies(pd.Series(regions.values))

In [83]:
reg_OHE.values.shape

(445026, 102)

In [84]:
data = data.join(reg_OHE)

In [85]:
from pandas.core.common import flatten

In [86]:
target = pd.DataFrame()

target['y+1'] = np.array([df[str(int(x))].values[1:-(5-1)] for x in forecast['region'].unique()]).flat
target['y+2'] = np.array([df[str(int(x))].values[2:-(5-2)] for x in forecast['region'].unique()]).flat
target['y+3'] = np.array([df[str(int(x))].values[3:-(5-3)] for x in forecast['region'].unique()]).flat
target['y+4'] = np.array([df[str(int(x))].values[4:-(5-4)] for x in forecast['region'].unique()]).flat
target['y+5'] = np.array([df[str(int(x))].values[5:] for x in forecast['region'].unique()]).flat
target['y+6'] = np.array([list(df[str(int(x))].values[6:]) + [np.nan] for x in forecast['region'].unique()]).flat

In [87]:
target

,y+1,y+2,y+3,y+4,y+5,y+6
0,44.0,29.0,19.0,17.0,16.0,14.0
1,29.0,19.0,17.0,16.0,14.0,6.0
2,19.0,17.0,16.0,14.0,6.0,7.0
3,17.0,16.0,14.0,6.0,7.0,12.0
4,16.0,14.0,6.0,7.0,12.0,7.0
...,...,...,...,...,...,...
445021,1.0,2.0,2.0,0.0,2.0,1.0
445022,2.0,2.0,0.0,2.0,1.0,2.0
445023,2.0,0.0,2.0,1.0,2.0,3.0
445024,0.0,2.0,1.0,2.0,3.0,9.0


In [39]:
data = data.join(target)

In [40]:
data = data.dropna()

In [41]:
print(data.columns.values)

['day' 'month' 'day_of_week' 'hour' 'c1' 's1' 'c2' 's2' 'c3' 's3' 'c4'
 's4' 'c5' 's5' 'c6' 's6' 'c7' 's7' 'c8' 's8' 'c9' 's9' 'c10' 's10' 'c11'
 's11' 'c12' 's12' 'c13' 's13' 'c14' 's14' 'c15' 's15' 'c16' 's16' 'c17'
 's17' 'c18' 's18' 'c19' 's19' 'c20' 's20' 'c21' 's21' 'y_f1' 'y_f2'
 'y_f3' 'y_f4' 'y_f5' 'y_f6' 'y_t' 'y_t-1' 'y_t-2' 'y_t-3' 'y_t-4' 'y_t-5'
 'y_t-6' 'y_t-24' 'y_t-48' 'y_t-72' 'y_t-96' 'y_t-120' 'y_t-144' 'y_t-168'
 1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174
 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224
 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278
 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333
 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385
 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441
 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068
 2069 2118 2119 2168 'y+1' 'y+2' 'y+3' 'y+4' 'y+5' 'y+6']


In [42]:
data

,day,month,day_of_week,hour,c1,s1,c2,s2,c3,s3,...,2069,2118,2119,2168,y+1,y+2,y+3,y+4,y+5,y+6
168,8,1,4,0,1.0,0.0,1.000000,-2.449294e-16,1.000000,-4.898587e-16,...,0,0,0,0,8.0,7.0,2.0,2.0,1.0,9.0
169,8,1,4,1,1.0,0.0,0.999301,3.739119e-02,0.997204,7.473009e-02,...,0,0,0,0,7.0,2.0,2.0,1.0,9.0,16.0
170,8,1,4,2,1.0,0.0,0.997204,7.473009e-02,0.988831,1.490423e-01,...,0,0,0,0,2.0,2.0,1.0,9.0,16.0,10.0
171,8,1,4,3,1.0,0.0,0.993712,1.119645e-01,0.974928,2.225209e-01,...,0,0,0,0,2.0,1.0,9.0,16.0,10.0,11.0
172,8,1,4,4,1.0,0.0,0.988831,1.490423e-01,0.955573,2.947552e-01,...,0,0,0,0,1.0,9.0,16.0,10.0,11.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445020,30,6,3,13,1.0,0.0,0.900969,-4.338837e-01,0.623490,-7.818315e-01,...,0,0,0,0,0.0,1.0,2.0,2.0,0.0,2.0
445021,30,6,3,14,1.0,0.0,0.916562,-3.998920e-01,0.680173,-7.330519e-01,...,0,0,0,0,1.0,2.0,2.0,0.0,2.0,1.0
445022,30,6,3,15,1.0,0.0,0.930874,-3.653410e-01,0.733052,-6.801727e-01,...,0,0,0,0,2.0,2.0,0.0,2.0,1.0,2.0
445023,30,6,3,16,1.0,0.0,0.943883,-3.302791e-01,0.781831,-6.234898e-01,...,0,0,0,0,2.0,0.0,2.0,1.0,2.0,3.0


In [43]:
start = datetime.datetime(2016,1,1,0)
apr = (datetime.datetime(2016,4,30,23) - start).total_seconds()/3600*102
may = (datetime.datetime(2016,5,31,23) - start).total_seconds()/3600*102

In [44]:
apr, may

(296106.0, 371994.0)

In [45]:
raw_apr_sample = data[data.month < 5]
raw_may_sample = data[data.month == 5]
raw_june_sample = data[data.month >= 6]

In [46]:
raw_june_sample

,day,month,day_of_week,hour,c1,s1,c2,s2,c3,s3,...,2069,2118,2119,2168,y+1,y+2,y+3,y+4,y+5,y+6
3648,1,6,2,0,1.0,0.0,-0.222521,-0.974928,-0.900969,0.433884,...,0,0,0,0,5.0,5.0,0.0,5.0,1.0,15.0
3649,1,6,2,1,1.0,0.0,-0.185912,-0.982566,-0.930874,0.365341,...,0,0,0,0,5.0,0.0,5.0,1.0,15.0,12.0
3650,1,6,2,2,1.0,0.0,-0.149042,-0.988831,-0.955573,0.294755,...,0,0,0,0,0.0,5.0,1.0,15.0,12.0,8.0
3651,1,6,2,3,1.0,0.0,-0.111964,-0.993712,-0.974928,0.222521,...,0,0,0,0,5.0,1.0,15.0,12.0,8.0,5.0
3652,1,6,2,4,1.0,0.0,-0.074730,-0.997204,-0.988831,0.149042,...,0,0,0,0,1.0,15.0,12.0,8.0,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445020,30,6,3,13,1.0,0.0,0.900969,-0.433884,0.623490,-0.781831,...,0,0,0,0,0.0,1.0,2.0,2.0,0.0,2.0
445021,30,6,3,14,1.0,0.0,0.916562,-0.399892,0.680173,-0.733052,...,0,0,0,0,1.0,2.0,2.0,0.0,2.0,1.0
445022,30,6,3,15,1.0,0.0,0.930874,-0.365341,0.733052,-0.680173,...,0,0,0,0,2.0,2.0,0.0,2.0,1.0,2.0
445023,30,6,3,16,1.0,0.0,0.943883,-0.330279,0.781831,-0.623490,...,0,0,0,0,2.0,0.0,2.0,1.0,2.0,3.0


In [47]:
raw_june_sample.values[:, :-6]

array([[ 1.,  6.,  2., ...,  0.,  0.,  0.],
       [ 1.,  6.,  2., ...,  0.,  0.,  0.],
       [ 1.,  6.,  2., ...,  0.,  0.,  0.],
       ...,
       [30.,  6.,  3., ...,  0.,  0.,  0.],
       [30.,  6.,  3., ...,  0.,  0.,  0.],
       [30.,  6.,  3., ...,  0.,  0.,  0.]])

## Обучение прогнозных моделей и проверка их качества на валидационном датасете

###  Задание 2


In [ ]:
#pred_featuers[6].shape

In [ ]:
#y_reg[6].shape

In [ ]:
# Ваш код
models = []
for i in range(1, 7):
    print(i)
    #print(pred_featuers[i].shape[0]==y_reg[i].shape[0])
    x = raw_apr_sample.values[:, :-6]
    y = raw_apr_sample[f'y+{i}'].values
    m = xgb.XGBRegressor()
    m.fit(x, y)
    models.append(m)
    #m = xgb.train(pred_featuers[i].values, y_reg[i].values)
    

In [ ]:
pred1 = models[0].predict(raw_may_sample.values[:, :-6])
pred2 = models[1].predict(raw_may_sample.values[:, :-6])
pred3 = models[2].predict(raw_may_sample.values[:, :-6])
pred4 = models[3].predict(raw_may_sample.values[:, :-6])
pred5 = models[4].predict(raw_may_sample.values[:, :-6])
pred6 = models[5].predict(raw_may_sample.values[:, :-6])

pred = np.array([pred1, pred2, pred3, pred4, pred5, pred6,]).T
true_val = raw_may_sample.values[:, -6: ]
    
err = abs(np.array(true_val) - np.array(pred)).sum()

In [ ]:
err = err/(102*739*6)
err

## Подбор оптимальных значений гиперпараметров

###  Задание 3



In [ ]:
import hyperopt.hp as hp
from hyperopt import fmin, tpe, STATUS_OK, Trials

In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 10, 2),
        'reg_alpha' : hp.quniform('reg_alpha', 50,150,25),
         'n_estimators': hp.quniform("n_estimators", 75, 110, 50),
        'seed': 0
    }

In [ ]:
def objective(space):
    
    s, train, response_train, valid, response_valid = space
    
    regression=xgb.XGBRegressor(
                    n_estimators =int(s['n_estimators']), max_depth = int(s['max_depth']),reg_alpha = int(s['reg_alpha'])
                    #, gamma = space['gamma'],min_child_weight=int(space['min_child_weight']),colsample_bytree=space['colsample_bytree'],
                    )
    
    evaluation = [( train, response_train), ( valid, response_valid)]
    
    regression.fit(train,response_train,
            eval_set=evaluation, eval_metric="mae",
            early_stopping_rounds=10,verbose=False)
    

    pred = regression.predict(valid)
    mae_pred = mae(response_valid,pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [ ]:
trials = Trials()

In [ ]:
best_hyperparams = []

Произведите поиск оптимальных значений гиперпараметров с помощью функции `fmin`

In [ ]:
for i in range(2, 7):
    trials = Trials()
    print(i)
    train = raw_apr_sample.values[:, :-6]
    response_train = raw_apr_sample[f'y+{i}'].values
    valid = raw_may_sample.values[:, :-6]
    response_valid = raw_may_sample[f'y+{i}'].values
    best_hyperparam = fmin(fn = objective,
                            space = [space, train, response_train, valid, response_valid,],
                            algo = tpe.suggest,
                            max_evals = 30,
                            trials = trials)
    best_hyperparams.append(best_hyperparam)

In [ ]:
print(best_hyperparams)

In [48]:
best_hyperparams = [{'max_depth': 6.0, 'n_estimators': 100.0, 'reg_alpha': 150.0}, {'max_depth': 8.0, 'n_estimators': 100.0, 'reg_alpha': 100.0}, {'max_depth': 8.0, 'n_estimators': 100.0, 'reg_alpha': 150.0}, {'max_depth': 8.0, 'n_estimators': 100.0, 'reg_alpha': 75.0}, {'max_depth': 8.0, 'n_estimators': 100.0, 'reg_alpha': 100.0}, {'max_depth': 10.0, 'n_estimators': 100.0, 'reg_alpha': 150.0}]

Обучите регрессию с оптимальным гиперпаметрами и  оцените ее качество на валидационном и тестовом датасетах

In [49]:
# ваш код
   # Ваш код
models_tuned = []
for i in range(1, 7):
    print(i)
    x = raw_apr_sample.values[:, :-6]
    y = raw_apr_sample[f'y+{i}'].values
    p = best_hyperparams[i-1]
    m = xgb.XGBRegressor(n_estimators =int(p['n_estimators']),
                         max_depth = int(p['max_depth']),
                         reg_alpha = int(p['reg_alpha']))
    m.fit(x, y)
    models_tuned.append(m)

    

1
2
3
4
5
6


###  Задание 4


In [91]:
pred1 = models_tuned[0].predict(raw_may_sample.values[:, :-6])
pred2 = models_tuned[1].predict(raw_may_sample.values[:, :-6])
pred3 = models_tuned[2].predict(raw_may_sample.values[:, :-6])
pred4 = models_tuned[3].predict(raw_may_sample.values[:, :-6])
pred5 = models_tuned[4].predict(raw_may_sample.values[:, :-6])
pred6 = models_tuned[5].predict(raw_may_sample.values[:, :-6])

pred = np.array([pred1, pred2, pred3, pred4, pred5, pred6,]).T
true_val = raw_may_sample.values[:, -6: ]
    
err = abs(np.array(true_val) - np.array(pred)).mean()

err

17.331127069440075

In [92]:
pred1 = models_tuned[0].predict(raw_june_sample.values[:, :-6])
pred2 = models_tuned[1].predict(raw_june_sample.values[:, :-6])
pred3 = models_tuned[2].predict(raw_june_sample.values[:, :-6])
pred4 = models_tuned[3].predict(raw_june_sample.values[:, :-6])
pred5 = models_tuned[4].predict(raw_june_sample.values[:, :-6])
pred6 = models_tuned[5].predict(raw_june_sample.values[:, :-6])

pred = np.array([pred1, pred2, pred3, pred4, pred5, pred6,]).T
true_val = raw_june_sample.values[:, -6: ]
    
err = abs(np.array(true_val) - np.array(pred)).mean()

err

17.558963414084783

In [ ]:
# Ошибка модели регрессии меньше модели SARIMA 17.5863 < 20.0054

In [90]:
df

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,116.0,190.0,135.0,132.0,395.0,308.0,401.0,336.0,496.0,260.0,...,2.0,44.0,4.0,297.0,311.0,104.0,9.0,142.0,96.0,1.0
2016-06-30 20:00:00,104.0,142.0,149.0,141.0,333.0,368.0,390.0,385.0,560.0,247.0,...,1.0,27.0,7.0,288.0,344.0,103.0,24.0,209.0,145.0,0.0
2016-06-30 21:00:00,151.0,162.0,145.0,135.0,359.0,422.0,460.0,541.0,672.0,259.0,...,2.0,21.0,9.0,287.0,307.0,185.0,9.0,213.0,142.0,1.0


In [88]:
data

,day,month,day_of_week,hour,c1,s1,c2,s2,c3,s3,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
0,1,1,4,0,1.0,0.0,1.000000,0.000000,1.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,1,1,4,1,1.0,0.0,0.999301,0.037391,0.997204,0.074730,...,0,0,0,0,0,0,0,0,0,0
2,1,1,4,2,1.0,0.0,0.997204,0.074730,0.988831,0.149042,...,0,0,0,0,0,0,0,0,0,0
3,1,1,4,3,1.0,0.0,0.993712,0.111964,0.974928,0.222521,...,0,0,0,0,0,0,0,0,0,0
4,1,1,4,4,1.0,0.0,0.988831,0.149042,0.955573,0.294755,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445021,30,6,3,14,1.0,0.0,0.916562,-0.399892,0.680173,-0.733052,...,1,0,0,0,0,0,0,0,0,0
445022,30,6,3,15,1.0,0.0,0.930874,-0.365341,0.733052,-0.680173,...,1,0,0,0,0,0,0,0,0,0
445023,30,6,3,16,1.0,0.0,0.943883,-0.330279,0.781831,-0.623490,...,1,0,0,0,0,0,0,0,0,0
445024,30,6,3,17,1.0,0.0,0.955573,-0.294755,0.826239,-0.563320,...,1,0,0,0,0,0,0,0,0,0


In [89]:
regions

time
2016-01-01 00:00:00    1435
2016-01-01 01:00:00    1435
2016-01-01 02:00:00    1435
2016-01-01 03:00:00    1435
2016-01-01 04:00:00    1435
                       ... 
2016-06-30 14:00:00    1630
2016-06-30 15:00:00    1630
2016-06-30 16:00:00    1630
2016-06-30 17:00:00    1630
2016-06-30 18:00:00    1630
Name: region, Length: 445026, dtype: int64

In [109]:
tmp

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,24.212280,63.638054,56.953835,29.193291,27.487831,12.518467,23.668196,35.830921,33.025543,12.518467,...,1406.438843,709.834290,399.679016,582.340454,42.977940,47.022839,29.193291,25.174040,20.375328,22.188829
1,37.385750,118.274956,104.532959,45.097546,34.251060,35.451942,42.573734,68.611763,36.391338,22.905294,...,1288.070435,696.051208,479.801666,566.096130,29.007368,83.688988,44.457317,38.537128,13.919923,16.039804
2,34.264030,101.873207,107.640427,53.518490,44.444855,37.073669,57.168137,80.284653,50.614384,25.683935,...,1157.241089,660.070129,493.401459,555.973999,52.544006,116.484077,38.436703,48.202965,23.684904,34.300552
3,27.400034,82.761177,94.869804,50.855450,52.255287,31.953405,45.483097,55.535011,43.269398,19.220924,...,808.831360,571.544128,684.118408,446.982513,63.850456,73.597015,28.201839,50.353348,29.354017,44.857029
4,18.262054,65.952698,78.646042,44.922642,38.029503,40.282566,37.851990,43.177063,40.302547,9.473447,...,313.900055,358.665222,550.422180,236.838181,29.099049,57.457664,28.981123,47.122257,28.623726,33.774494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4358,7.278799,24.173317,23.328156,8.007952,3.562763,3.583373,6.226185,6.613765,15.877254,4.903563,...,618.047546,819.366699,797.365540,613.764832,5.946981,4.106265,6.345374,4.106265,2.551227,1.578640
4359,6.526605,24.840553,24.980507,7.314868,3.946335,2.110102,4.065791,5.078123,10.939371,4.410124,...,534.506165,538.804504,634.225525,481.302399,4.484466,2.832359,5.578584,2.520812,1.711709,1.345159
4360,7.319644,35.187511,36.991802,16.395649,7.079475,3.177557,7.937601,9.902601,16.354164,7.239051,...,707.461243,567.129150,751.159851,629.088867,5.807927,4.403341,6.843871,4.251275,2.613649,2.318086
4361,9.765262,37.715744,38.245770,15.901551,7.892891,4.251275,8.126925,16.906538,26.196146,12.732061,...,971.296448,828.244385,1027.140747,836.459106,7.307661,5.718488,6.845444,4.834520,2.716479,2.507409


In [129]:
df.columns = list(map(lambda x: str(x), df.columns))

In [134]:
df = df.iloc[:-5]

In [140]:
regions

time
2016-01-01 00:00:00    1435
2016-01-01 01:00:00    1435
2016-01-01 02:00:00    1435
2016-01-01 03:00:00    1435
2016-01-01 04:00:00    1435
                       ... 
2016-06-30 14:00:00    1630
2016-06-30 15:00:00    1630
2016-06-30 16:00:00    1630
2016-06-30 17:00:00    1630
2016-06-30 18:00:00    1630
Name: region, Length: 445026, dtype: int64

In [135]:
for i in range(6):
    pred = models_tuned[i].predict(data.values)
    pred_by_reg = np.array(np.split(pred, 102)).T
    col = list(map(lambda x: str(x) + '_' + str(i+1), regions.unique()))
    tmp = pd.DataFrame(pred_by_reg)
    tmp.columns = col
    tmp.index = df.index
    df = df.join(tmp)

In [139]:
print(df.columns.values)

['1075' '1076' '1077' '1125' '1126' '1127' '1128' '1129' '1130' '1131'
 '1132' '1172' '1173' '1174' '1175' '1176' '1177' '1178' '1179' '1180'
 '1181' '1182' '1183' '1184' '1221' '1222' '1223' '1224' '1225' '1227'
 '1228' '1229' '1230' '1231' '1232' '1233' '1234' '1235' '1272' '1273'
 '1274' '1278' '1279' '1280' '1281' '1282' '1283' '1284' '1285' '1286'
 '1287' '1326' '1327' '1331' '1332' '1333' '1334' '1335' '1336' '1337'
 '1338' '1339' '1376' '1377' '1378' '1380' '1382' '1383' '1384' '1385'
 '1386' '1387' '1388' '1389' '1390' '1426' '1431' '1434' '1435' '1436'
 '1437' '1438' '1439' '1441' '1442' '1480' '1482' '1483' '1530' '1532'
 '1533' '1580' '1630' '1684' '1733' '1734' '1783' '2068' '2069' '2118'
 '2119' '2168' '1435_1' '1436_1' '1437_1' '1438_1' '1439_1' '1441_1'
 '1442_1' '1173_1' '1174_1' '1175_1' '1182_1' '1224_1' '1225_1' '1129_1'
 '1130_1' '1176_1' '1177_1' '1178_1' '1221_1' '1222_1' '1223_1' '1227_1'
 '1228_1' '1272_1' '1273_1' '1274_1' '1278_1' '1326_1' '1327_1' '1376_1'
 '

In [144]:
df.to_csv('./pred.csv')